 - Прочитать про методы оптимизации для нейронных сетей https://habr.com/post/318970/

 - Реализовать самостоятельно логистическую регрессию

   - Обучить ее методом градиентного спуска
   - Методом nesterov momentum
   - Методом rmsprop
 - В качестве dataset’а взять Iris, оставив 2 класса:

   - Iris Versicolor
   - Iris Virginica

In [1]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import sys

В данном задании был выбран более сложный алгоритм логистической регресии через функцию softmax, которая позволяет работать с произвольным количеством предсказываемых классов. Для каждого класса ведется свой набор коэффициентов, в результате имеем не (n,) подбираемых коофициентов, а матрицу (k,n) коэффициентов, k-тая строка описывает k-тый класс.

In [2]:
class CustomLogisticRegression(BaseEstimator, ClassifierMixin):
    """
    Параметры:
    ----------
      max_iter: количество итераций алгоритма градиентного спуска
      solver: используемый алгоритм, возможные значения "sgd","nesterov","rmsprop"
      eta: скорость обучения
      gamma: гиперпараметр для агоритмов nesterov и rmsprop
      epsilon: гиперпарметр алгоритма rmsprop
      verbose: вывод отладочной информации при работе алгоритма
    """
    def __init__(self, max_iter=1000, solver="sgd", eta=0.01, gamma=0.9, epsilon=10e-8, verbose=False):
        self.max_iter = max_iter
        if solver == "sgd":
          self.solver = solver, None, None, self._sgd
        elif solver == "nesterov":
          self.solver = solver, self._nesterov_init, self._nesterov_get_theta_for_gradient_calc, self._nesterov
        elif solver == "rmsprop":
          self.solver = solver, self._rmsprop_init, None, self._rmsprop
        else:
          raise ValueError("Invalid solver '%s' specified." % solver)
        self.eta = eta
        self.gamma = gamma
        self.epsilon = epsilon
        self.verbose = verbose

    def _sgd(self, theta, gradients):
      # theta, gradients - (k,n)
      return theta - self.eta * gradients

    def _nesterov_init(self, k, n):
      self.vt_prev = np.zeros((k, n))

    def _nesterov_get_theta_for_gradient_calc(self, theta):
      return theta - self.gamma * self.vt_prev
      
    def _nesterov(self, theta, gradients):
      vt = self.gamma * self.vt_prev + self.eta * gradients
      self.vt_prev = vt
      return theta - vt

    def _rmsprop_init(self, k, n):
      self.EJ2_prev = np.zeros((k, n))

    def _rmsprop(self, theta, gradients):
      EJ2 = self.gamma * self.EJ2_prev + (1 - self.gamma) * gradients * gradients
      self.EJ2_prev = EJ2
      return theta - self.eta * gradients / np.sqrt(EJ2 + self.epsilon)

    def _print(self, msg, *msg_args):
        if not self.verbose:
            return
        if self.verbose < 50:
            writer = sys.stderr.write
        else:
            writer = sys.stdout.write
        writer(msg.format(*msg_args) + "\n")

    def fit(self, X, y):
      assert len(X.shape) == 2 and len(y.shape) == 1
      assert X.shape[0] == y.shape[0]

      # M - кол-во образцов, N - кол-во признаков
      M, N = X.shape

      # K - количество классов
      self.class_values = np.unique(y)
      K = len(self.class_values)

      # обнуляем временные переменные регуляризации если они нужны
      if self.solver[1] is not None:
        self.solver[1](K, N)

      # подготавливаем (k,n) матрицу коэффициентов, каждая строка представляет коэффициенты k-го класса
      theta = np.random.randn(K, N)

      self._print("Старт обучения, solver '{}', theta:\n{}\n", self.solver[0], theta)

      # yk = 1 - если целевым классом для i-го образца является k, иначе 0 -> (k,m)
      yk = np.array([[1 if y[m] == self.class_values[k] else 0 for m in range(M)] for k in range(K)])

      # далее цикл градиентного спуска
      for itertion in range(self.max_iter):

        if self.solver[2] is not None:
          theta = self.solver[2](theta)

        #  Sk(x) = theta * X
        # theta - (k,n), X = (m,n), skx = (k,m)
        skx = np.dot(theta, X.T)

        # exp(Sk(x)) -> (k,m)
        eskx = np.exp(skx)

        # sum(exp(Sk(x))) -> (m,)
        sum_eskx = eskx.sum(axis=0)

        # Pk = exp(Sk(x))/sum(exp(Sk(x))) -> (k,m)
        pk = eskx / sum_eskx

        # считаем векторы-градиенты перекрестной энтропии (функции издержек) для классов -> (k,n)
        gradients = np.dot(pk - yk, X) / M

        # движемся по гиперповерхности
        theta = self.solver[3](theta, gradients)

        if itertion % 100 == 0 :
          self._print("Итерация {} theta:\n{}\n", itertion, theta)

      self.theta = theta
      self._print("Конец обучения. theta:\n{}\n", theta)

    def predict_proba(self, X):
      assert len(X.shape) == 2 and X.shape[1] == self.theta.shape[1]
      
      skx = np.dot(self.theta, X.T)        # -> (k,d)
      eskx = np.exp(skx)          # -> (k,d)
      sum_eskx = eskx.sum(axis=0) # -> (d,)
      pk = eskx / sum_eskx        # -> (k,d)
      return pk.T

    def predict(self, X):
      assert len(X.shape) == 2 and X.shape[1] == self.theta.shape[1]
      # theta - (k,n), X - (d,n)  ->  (d,)

      skx = np.dot(self.theta, X.T)        # -> (k,d)
      y = np.argmax(skx, axis=0)  # -> (d,)
      return [self.class_values[v] for v in y]


In [3]:
data = load_iris()
X, y, feature_names, target_names = data.data, data.target, data.feature_names, data.target_names

In [4]:
np.random.seed(1)

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.2)

In [6]:
models = [
  ("Sequential Gradient Descent", CustomLogisticRegression(solver="sgd")),
  ("Nesterov momentum", CustomLogisticRegression(solver="nesterov")),
  ("Root Mean Square Propogation", CustomLogisticRegression(solver="rmsprop")),
  ("sklearn LogisticRegression", LogisticRegression(multi_class="multinomial", solver="lbfgs"))
]


In [7]:
results = []

for name, model in models:
  model.fit(X_train, y_train)

  y_pred = model.predict(X_val)

  score_train = model.score(X_train, y_train)
  score_test = model.score(X_val, y_val)
  cm = confusion_matrix(y_val, y_pred)

  results.append((name, score_train, score_test, cm))


In [8]:
for name, score_train, score_test, cm in results:
  print("------------------")
  print("Algorithm name:", name)
  print("Accuracy train:", score_train)
  print("Accuracy test:", score_test)
  print("Confusion matrix:\n", cm)

------------------
Algorithm name: Sequential Gradient Descent
Accuracy train: 0.8666666666666667
Accuracy test: 0.825
Confusion matrix:
 [[40  0  0]
 [ 0 36  1]
 [ 0 20 23]]
------------------
Algorithm name: Nesterov momentum
Accuracy train: 1.0
Accuracy test: 0.9583333333333334
Confusion matrix:
 [[40  0  0]
 [ 0 33  4]
 [ 0  1 42]]
------------------
Algorithm name: Root Mean Square Propogation
Accuracy train: 1.0
Accuracy test: 0.9666666666666667
Confusion matrix:
 [[40  0  0]
 [ 0 34  3]
 [ 0  1 42]]
------------------
Algorithm name: sklearn LogisticRegression
Accuracy train: 0.9666666666666667
Accuracy test: 0.9333333333333333
Confusion matrix:
 [[40  0  0]
 [ 0 36  1]
 [ 0  7 36]]
